In [ ]:
import pandas as pd
import numpy as np
from global_functions import get_newest_date_file

In [ ]:
df_2020 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\BASE_YEAR\2020_jtmt_forcast_full_240506.xlsx").set_index('Taz_num').sort_index() 
df_jtmt_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_jtmt.xlsx").set_index('Taz_num').sort_index() 
df_iplan_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_iplan.xlsx").set_index('Taz_num').sort_index() 
df_bau_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_bau.xlsx").set_index('Taz_num').sort_index() 

In [ ]:
BaseProjections_version = get_newest_date_file(r'C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\JTMT', 'BaseProjections2025_')

years = ['2025', '2030','2035', '2040', '2045', '2050']

senarios = ['JTMT', 'IPLAN', 'BAU']

for sen in senarios:
    for year in years:
        locals()[f'BaseProjections_{sen}_{year}'] = pd.read_excel(fr'C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\{sen}\BaseProjections{year}_{BaseProjections_version}_{sen.lower()}.xlsx')
        locals()[f'values_list_{sen}'] = ['NaN']

In [ ]:
pd.options.display.max_columns = None

In [ ]:
df_jtmt_2050 = pd.merge(df_jtmt_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')
df_iplan_2050 = pd.merge(df_iplan_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')
df_bau_2050 = pd.merge(df_bau_2050,df_2020[['SCHOOLDIST']],on='Taz_num', how='left')

In [ ]:
def convert_to_int(float_list):
  int_list = []
  for num in float_list:
    if not isinstance(num, float):
      raise ValueError("Input list must contain only floats")
    # Check if conversion to int results in a loss of precision
    if num != int(num):
      raise ValueError(f"Float {num} cannot be converted to int without losing precision")
    int_list.append(int(num))
  return int_list

In [ ]:
def add_total_by_type_row(output_df, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df[df[f'{column_input_name}'] == column_input_value] 
        values_list.append(len(val))

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [ ]:
def add_total_row(output_df, column_input_name, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if column_input_name in df.columns:
            val = df[f'{column_input_name}'].sum()
            # val = df.pivot_table(index='main_secto', values=f'{column_input_name}', aggfunc="sum", fill_value=0).astype('float32').sum()
            values_list.append("{:,}".format(int(val)))
        else:
            values_list.append('NaN')

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [ ]:
def add_taz_num_is_unique_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df_2020.index.is_unique
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [ ]:
def add_taz_num_count_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = len(df.index)
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [ ]:
def add_percentage_growth_row(current, previous, output_df, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if len(list(df)) == 135:
                values_list.append('NaN')

        else:
            if df[f'{current}'].sum() == df[f'{previous}'].sum():
                values_list.append(100.0)
            try:
                values_list.append(int((df[f'{current}'].sum() / df[f'{previous}'].sum()) * 100.0))
            except ZeroDivisionError:
                values_list.append(0)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [ ]:
def add_students_by_SCHOOLDIST(output_df, column_key_name, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    # if column_key_name in df.columns:
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = pd.pivot_table(df, index=[f'{column_input_name}'], aggfunc="sum")
        if column_key_name in val.columns:
            values_list.append(val[column_key_name][column_input_value])
        else:
            values_list.append('NaN')

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [ ]:
monitoring_df = pd.DataFrame(columns=['2020', 'JTMT', 'iplan', 'bau'])

In [ ]:
SCHOOLDIST_list = list(df_2020['SCHOOLDIST'].value_counts().index)
sectors_list = list(df_2020['main_secto'].value_counts().index)
zonetypes_list = list(df_2020['zonetype'].value_counts().index)
jeru_metros_list = list(df_2020['jeru_metro'].value_counts().index)
in_jerusals_list = list(df_2020['in_jerusal'].value_counts().index)

In [ ]:
# אוכלוסיה
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva', 'pop_2020')
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva_2050', 'pop_2050')

# מועסקים
monitoring_df = add_total_row(monitoring_df, 'total_emp', 'emp_2020')
monitoring_df = add_total_row(monitoring_df, 'total_emp_2050', 'emp_2050')

# מספר א"ת לפי סקטור
for item in sectors_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', item, f'Taz_U_{item}')

# סטודנטים
monitoring_df = add_total_row(monitoring_df, 'univ', 'univ_2020')
monitoring_df = add_total_row(monitoring_df, 'univ_2050', 'univ_2050')

# בתי ספר
monitoring_df = add_total_row(monitoring_df, 'student', 'student_2020')
monitoring_df = add_total_row(monitoring_df, 'student_2050', 'student_2050')

# ישיבות
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim', 'student_yeshiva_and_kollim_2020')
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim_2050', 'student_yeshiva_and_kollim_2050')

# zonetype
for item in zonetypes_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'zonetype', item, f'zonetype_{item}')

# jeru_metro
for item in jeru_metros_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'jeru_metro', item, f'jeru_metro_{item}')

# in_jerusal
for item in in_jerusals_list:
    monitoring_df = add_total_by_type_row(monitoring_df, 'in_jerusal', item, f'in_jerusal_{item}')

# אין א"ת שחוזר פעמיים
monitoring_df = add_taz_num_is_unique_row(monitoring_df, 'taz_num_is_unique')
# מספר א"ת
monitoring_df = add_taz_num_count_row(monitoring_df, 'taz_num_count')

In [ ]:
# גידול אוכלוסיה כל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2025', 'pop_without_dorms_yeshiva', monitoring_df, 'percentage growth 2020-2025')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2030', 'pop_without_dorms_yeshiva_2025', monitoring_df, 'percentage growth 2025-2030')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2035', 'pop_without_dorms_yeshiva_2030', monitoring_df, 'percentage growth 2030-2035')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2040', 'pop_without_dorms_yeshiva_2035', monitoring_df, 'percentage growth 2035-2040')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2045', 'pop_without_dorms_yeshiva_2050', monitoring_df, 'percentage growth 2045-2050')

# גידול סטודנטים כל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2025', 'pop_without_dorms_yeshiva', monitoring_df, 'percentage growth 2020-2025')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2030', 'pop_without_dorms_yeshiva_2025', monitoring_df, 'percentage growth 2025-2030')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2035', 'pop_without_dorms_yeshiva_2030', monitoring_df, 'percentage growth 2030-2035')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2040', 'pop_without_dorms_yeshiva_2035', monitoring_df, 'percentage growth 2035-2040')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2045', 'pop_without_dorms_yeshiva_2050', monitoring_df, 'percentage growth 2045-2050')

In [ ]:
# SCHOOLDIST 2020
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2020')

# SCHOOLDIST 2025
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2025', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2025')

# SCHOOLDIST 2030
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2030', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2030')

# SCHOOLDIST 2035
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2035', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2035')

# SCHOOLDIST 2040
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2040', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2040')

# SCHOOLDIST 2045
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2045', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2045')

# SCHOOLDIST 2050
for item in SCHOOLDIST_list:
    monitoring_df = add_students_by_SCHOOLDIST(monitoring_df, 'student_2050', 'SCHOOLDIST', item, f'SCHOOLDIST_{item}_2050')

In [ ]:
yearbooks = ['5_9', '10_14', '15_19']

values_list = ['NaN']

for yearbook in yearbooks:
    for year in years:
        for idx, sen in enumerate(senarios):
            locals()[f'values_list_{year}']  = ['NaN']
            values_list.append(locals()[f'BaseProjections_{senarios[idx]}_{year}'][f'age{yearbook}'].sum()/5*4)
        monitoring_df.loc[f'BaseProjections_{year}_age{yearbook}'] = values_list
        values_list = ['NaN']

In [ ]:
# monitoring_df.to_excel('monitoring_df.xlsx')
monitoring_df